# Habituation to weakly non-Gaussian backgrounds as a function of background scale and moments

Check how the mean, variance, and third moment of the concentrations affect the convergence time scale, using the $\epsilon$ parameters to control the third moment without changing the first two moments too much. 

We analyze uncoupled networks, because $\eta$ coupling changes individual neurons' initial conditions and dynamics, making it very hard to predict anything analytically beyond a rough scaling argument. So, in other words, we consider a single IBCM neuron. 

We can predict convergence in two phases: first, convergence to the saddle point, where $h_d = \langle c \rangle \sum_\gamma h_\gamma$ reaches approximately $1$, and second, exponential divergence from the saddle point, at the time scale predicted from the Jacobian matrix diagonalized numerically. 

The starting point is the equation for one of the alignments $h_\gamma$ averaged over fast background fluctuations, and neglecting temporal correlations between $\Theta$ and $\mathbf{m}$, 

$$ \frac{1}{\mu} \frac{\mathrm{d} h_\nu}{\mathrm{d} t} = \sum_\gamma \left[ \langle c \rangle (h_\mathrm{d}^2 +  \sigma^2 u^2)(1 - h_\mathrm{d}) - \sigma^2 h_\gamma (h_\mathrm{d}^2 + \sigma^2 u^2 - 2 h_\mathrm{d}) + m_3 h_\gamma^2 \right] \mathbf{\hat{s}}_\nu^T \mathbf{\hat{s}}_\gamma $$

where $\langle c \rangle$, $\sigma^2$, and $m_3$ are the average, variance, and third centered moment of the background concentration process. 

## Phase I
For phase I, the convergence to the saddle point can be analyzed by looking at the equation for $h_\mathrm{d}$, obtained by simming the above ODE over $\nu$ and multiplying by $\langle c \rangle$. For small $h_\mathrm{d}$, $u^2$, it takes the form $\mathrm{d}h_\mathrm{d}/\mathrm{d}t \sim h_\mathrm{d}^2$ . We can thus predict the convergence time from a divergence time of this purely second-order equation when neglecting the higher-order terms, resulting in a time of

$$ t_\mathrm{ds} = \frac1A \left(\frac{1}{h_\mathrm{d,0}} - 1 \right) $$

where $A = mu f_0 (N_\mathrm{B} \langle c \rangle^2 + 2 \sigma^2)$ and $f_0$ is the mean-field approximation to the sum of dot products between one odor vector with all others, $f_0 \approx \sum_{\gamma} \mathbf{\hat{s}}_\gamma^T \mathbf{\hat{s}}_0 \approx 1 + (N_\mathrm{B}-1) d_\mathrm{cos}$, with $d_\mathrm{cos}$ the average cosine similarity between two odor vectors in the ensemble. This $d_\mathrm{cos}$ is computed for $N_\mathrm{S}$ dimensions in the notebook. For odors with exponential elements and unit-normed, $d_\mathrm{cos} = 0.516 \pm 0.002$ for $N_\mathrm{S} = 50$ dimensions, $d_\mathrm{cos} = 0.531 \pm 0.002$ for $N_\mathrm{S} = 25$ dimensions. 

## Phase II
We use the largest eigenvalue of the Jacobian matrix computed at the saddle point. We use our analytical predictions of IBCM fixed points for this, and compute the eigenvalues numerically (not much of a choice), the matrix is large. 

We look at the value of $u^2$ in phase II, since it encompasses the specific $h_\nu$ converging to $h_\mathrm{sp}$ and diverging away from the other $h_\nu'$ which go to the non-specific value $h_\mathrm{ns}$. We compute the convergence time by asking how long it takes for this exponential exit from the saddle to reach a target $u^2$ value away from the value at the saddle $u^2_\mathrm{saddle}$, starting from an initial $u^2$ value near the saddle, at $u^2_\mathrm{saddle} + \Delta u^2_s$ (because the solution never lands perfectly on the saddle, it approaches near it). This exponential increases in $u^2$ has the form

$$ u^2(t) = u^2_\mathrm{saddle} +  \Delta u^2_s e^{(t - t_{ds})/\tau_\mathrm{jac}} $$

where $\tau_\mathrm{jac}$ is the inverse of the largest eigenvalue real part of the Jacobian matrix, and this exponential increase starts at time $t_{ds}$ (end of phase 1) near the saddle. We solve for the time $t_u$ it takes for $u^2$ to reach a target value below its fixed point value; $u^2_\mathrm{target} = 1/\sigma^2$ is a good choice because it is below the fixed point for any $\epsilon > 0$ (i.e. any background with a non-zero third moment). This gives a time

$$ t_u = t_{ds} + \tau_\mathrm{jac} \log \left(\frac{u^2_\mathrm{target}-u^2_\mathrm{saddle}}{\Delta u^2}  \right) $$

## Imports

In [ ]:
import numpy as np
from scipy import sparse, special
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from time import perf_counter
import os, json
from os.path import join as pj
import sys
if ".." not in sys.path:
    sys.path.insert(1, "..")


from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_hgammas_hbargammas,
    ibcm_respond_new_odors
)
from modelfcts.ibcm_analytics import (
    fixedpoint_thirdmoment_exact, 
    ibcm_fixedpoint_w_thirdmoment, 
    ibcm_all_largest_eigenvalues,
    jacobian_fixedpoint_thirdmoment
)
from modelfcts.biopca import (
    integrate_inhib_biopca_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import (
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor, 
    compute_ideal_factor
)
from modelfcts.checktools import (
    analyze_pca_learning, 
)
from modelfcts.backgrounds import (
    update_thirdmoment_kinputs, 
    sample_ss_distrib_thirdmoment, 
    generate_odorant
)
from utils.statistics import seed_from_gen
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_hbars_gamma_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard, l2_norm

# Initialization

### Aesthetic parameters

In [ ]:
do_save_plots = False
do_save_outputs = False

root_dir = pj("..")
outputs_folder = pj(root_dir, "results", "for_plots", "convergence")
panels_folder = pj(root_dir, "figures", "convergence")
params_folder = pj(root_dir, "results", "common_params")

# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

# Background generation and initialization functions

In [ ]:
def linear_combi(concs, backs):
    """ concs: shaped [..., n_odors]
        backs: 2D array, shaped [n_odors, n_osn]
    """
    return concs.dot(backs)

In [ ]:
# Global choice of background and odor mixing functions
update_fct = update_thirdmoment_kinputs
combine_fct = linear_combi

In [ ]:
# Given epsilon and the first three moment parameters, 
# compute all related background update parameter (matrices e^A, e^B, etc.)
def default_nongauss_background_params(n_comp, avgnu, sigma2, epsilon_nu, 
                                       tau_nu, dt=1.0, correl_rho=0.0):
    """ Build background parameter lists from :
    Args:
        n_comp: number of background vectors
        avgnu: average concentration, default is 1/sqrt(n_comp)
        sigma2: variance
        epsilon_nu: controls the third moment amplitude, epsilon*sigma^4
        tau_nu: autocorrelation time of the underlying O-U process
        dt: Euler time steps, in units of 10 ms (default is 1.0)
        correl_rho: correlation between odors, default 0.0
    """
    
    # Initial background vector and initial nu values
    averages_nu = np.full(n_comp, avgnu)

    ## Compute the matrices in the Ornstein-Uhlenbeck update equation
    # Update matrix for the mean term: 
    # Exponential decay with time scale tau_nu over time deltat
    update_mat_A = np.identity(n_comp)*np.exp(-dt/tau_nu)

    # Steady-state covariance matrix
    steady_covmat = correl_rho * sigma2 * np.ones([n_comp, n_comp])  # Off-diagonals: rho
    steady_covmat[np.eye(n_comp, dtype=bool)] = sigma2  # diagonal: ones

    # Cholesky decomposition of steady_covmat gives sqrt(tau/2) B
    # Update matrix for the noise term: \sqrt(tau/2(1 - exp(-2*deltat/tau))) B
    psi_mat = np.linalg.cholesky(steady_covmat)
    update_mat_B = np.sqrt(1.0 - np.exp(-2.0*dt/tau_nu)) * psi_mat

    back_params = [update_mat_A, update_mat_B, None, averages_nu, epsilon_nu]
    return back_params

In [ ]:
# Background initialization, given parameters and a seeded random generator
def initialize_given_background(back_pms, rgen, n_comp):
    # Initial values of background process variables (t, c for each variable)
    init_nu = np.zeros(n_comp)
    back_comp = back_pms[2]
    init_bkvec = combine_fct(init_nu, back_comp)
    init_back = [init_nu, init_bkvec]
    return init_back

# IBCM simulation functions

In [ ]:
# Analyze to establish convergence to specific fixed points. 
def analyze_ibcm_simulation(sim_results, ibcm_rates_loc, back_pms, 
                            skp_loc=20, dt=1.0, duration_loc=360000.0):
    """
    Args:
        sim_results = (tser_ibcm, nuser_ibcm, bkvecser_ibcm, mser_ibcm, 
            hbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm)
        ibcm_rates_loc: learnrate_ibcm, tau_avg_ibcm, coupling_eta_ibcm, ...
            
    Returns:
        alignment_gaps, indexed [neuron]
        specif_gammas, indexed [neuron]
        gamma_vari, indexed [neuron, component]
    """
    coupling = ibcm_rates_loc[2]
    (tser_ibcm, nuser_ibcm, bkvecser_ibcm, mser_ibcm, 
        hbarser_ibcm, thetaser_ibcm, wser_ibcm, yser_ibcm) = sim_results
    # Calculate hgammas_bar and mbars
    transient = int(5/6*duration_loc / dt) // skp_loc
    basis = back_pms[2]
    
    # Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
    mbarser, c_gammas, hbars_gamma = compute_mbars_hgammas_hbargammas(mser_ibcm, coupling, basis)
    hbars_gamma_mean = np.mean(hbars_gamma[transient:], axis=0)
    # Sorted odor indices, from min to max, of odor alignments for each neuron
    aligns_idx_sorted = np.argsort(hbars_gamma_mean, axis=1) 
    specif_gammas = np.argmax(hbars_gamma_mean, axis=1)
    assert np.all(specif_gammas == aligns_idx_sorted[:, -1])
    
    
    # Gap between first and second largest alignments for each neuron
    n_i = hbars_gamma_mean.shape[0]
    alignment_gaps = (hbars_gamma_mean[np.arange(n_i), specif_gammas]
                     - hbars_gamma_mean[np.arange(n_i), aligns_idx_sorted[:, -2]])
    
    # Variance (fluctuations) of hbars gamma in the last 20 minutes of the simul
    # Increases when the learning rate increases
    last_steps = int(2.0*duration_loc/3.0 / dt) // skp_loc
    hbars_gamma_vari = np.var(hbars_gamma[last_steps:], axis=0)
    
    return hbars_gamma, alignment_gaps, specif_gammas, hbars_gamma_vari

In [ ]:
def run_analyze_ibcm_one_back_seed_nongauss(
        ibcm_rates_loc, back_rates, inhib_rates_loc, 
        options_loc, dimensions, seedseq, init_ampli=0.001,
        duration_loc=360000.0, dt_loc=1.0, skp_loc=20, full_returns=False
    ):
    """ Given IBCM model rates and background parameters except
    background odors (but incl. number odors, c0), and a main seed sequence, 
    run and analyze convergence of IBCM on the background generated from that seed. 
    The seedseq should itself have been spawned from a root seed to have a distinct
    one per run; this still makes seeds reproducible yet distinct for different runs. 
    The seedseq here is spawned again for a background gen. seed and a simul. seed. 
    
    Args:
        dimensions: gives [n_components, n_dimensions, n_i_ibcm]
    
    Returns:
        iff full_return:
            gaps, specifs, hgamvari, hgammas_ser, sim_results
        else:
            gaps, specifs, hgamvari, None, None
        alignment_gaps: indexed [neuron]
        specif_gammas: indexed [neuron]
        gamma_vari: indexed [neuron, component]
    """
    #print("Initializing IBCM simulation...")
    # Get dimensions
    n_comp, n_dim, n_i = dimensions
    
    # Spawn back. generation seed and simul seed
    initseed, simseed = seedseq.spawn(2)
    
    # Duplicate back params before including locally-generated odor vectors to them
    back_pms_loc = list(back_rates)
    
    # Create background
    rgen_init = np.random.default_rng(initseed)
    back_comps_loc = generate_odorant((n_comp, n_dim), rgen_init)
    back_comps_loc = back_comps_loc / l2_norm(back_comps_loc, axis=1)[:, None]

    # Put generated odors in the list of background parameters
    back_pms_loc[2] = back_comps_loc

    # Initialize background with the random generator with seed rgenseed
    rgen_init = np.random.default_rng(initseed)
    init_back = initialize_given_background(back_pms_loc, rgen_init, n_comp)

    # Initial synaptic weights: small positive noise
    lambd_loc = ibcm_rates_loc[3]
    # Random initial magnitudes
    init_magnitudes = init_ampli * 0.5 * (1.0 + rgen_init.random(size=[n_i, 1]))*lambd_loc
    init_synapses_ibcm = init_magnitudes * np.abs(rgen_init.normal(size=[n_i, n_dim]))
    
    # Run the IBCM simulation
    #print("Running IBCM simulation...")
    tstart = perf_counter()
    sim_results = integrate_inhib_ibcm_network_options(
                init_synapses_ibcm, update_fct, init_back, 
                ibcm_rates_loc, inhib_rates_loc, back_pms_loc, 
                duration_loc, dt_loc, seed=simseed, 
                noisetype="normal",  skp=skp_loc, **options_loc
    )
    tend = perf_counter()
    #print("Finished IBCM simulation in {:.2f} s".format(tend - tstart))
    # IBCM results: [tseries, bk_series, bkvec_series, m_series,
    #        cbar_series, theta_series, w_series, y_series]
    
    # Now analyze IBCM simul for convergence
    #print("Starting to analyze IBCM simulation...")
    tstart = perf_counter()
    hgammas_ser, gaps, specifs, hgamvari = analyze_ibcm_simulation(sim_results, 
                        ibcm_rates_loc, back_pms_loc, skp_loc=skp_loc, duration_loc=duration_loc)
    tend = perf_counter()
    #print("Finished analyzing IBCM simulation")
    
    # Doesn't return full c gamma series, only the summary statistics of convergence
    if full_returns:
        hgammas_ser_ret = hgammas_ser
        sim_results_ret = sim_results
        back_comps_loc_ret = back_comps_loc
    else:
        hgammas_ser_ret = None
        sim_results_ret = None
        back_comps_loc_ret = None
    
    return gaps, specifs, hgamvari, hgammas_ser_ret, sim_results_ret, back_comps_loc_ret

Enjoy a simplification for once: we do not need to consider other models like average subtraction, optimal $P$, orthogonal projection since all we care about in this notebook is the convergence of the two biologically plausible models. 

# Plotting functions

In [ ]:
# Plotting functions for IBCM
def plot_ibcm_results(res_ibcm_raw, hbars_gamma, skp=20, dtscale=10.0 / 60.0 / 1000.0):
    # tseries, bk_series, bkvec_series, m_series,
    # hbar_series, theta_series, w_series, y_series
    tser, bkser, bkvecser, _, _, _, _, yser = res_ibcm_raw
    tser_scaled = tser * dtscale  # in min
    # but will multiply by 1000 to compensate /1000 in plotting functions
    # Plot of hbars gamma series
    fig , ax, _ = plot_hbars_gamma_series(tser_scaled*1000, hbars_gamma,
                            skp=skp, transient=320000 // skp)
    fig.tight_layout()
    leg = ax.legend(loc="upper left", bbox_to_anchor=(1., 1.))
    ax.set_xlabel("Time (min)")
    plt.show()
    plt.close()

    # Plot of background inhibition
    fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                    tser_scaled*1000, bkvecser, yser, skp=skp)
    ax.set_xlabel("Time (min)")

    # Compute noise reduction factor, annotate
    transient = 250000 // skp
    norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

    print("Mean activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
    print("Standard deviation of activity norm reduced to "
          + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
    ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
               xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

    ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
    fig.tight_layout()
    
    plt.show()
    plt.close()

# Simulation parameter choices

## Simulation parameters common to all runs

In [ ]:
# Common parameters for all simulations
# Dimensions: 25 is enough?
n_dimensions = 25
n_components = 3  # try with 3 for simplicity by default

# Inhibition W learning and decay rates
inhib_rates_default = [0.0001, 0.00002]  # alpha, beta  [0.00025, 0.00005]

# Simulation duration and integration time step
duration = 360000.0
deltat = 1.0  # time step units, each is 10 ms
dtscale = 10.0 / 1000.0 / 60.0  # convert time steps to minutes

# Saving every skp simulation point, 50 is enough for plots, 
# here use 20 to get convergence time accurately
skp_default = 20 * int(1.0 / deltat)
tser_common = np.arange(0.0, duration, deltat*skp_default)

# Common model options
activ_function = "identity"  #"ReLU"

## IBCM default parameters

In [ ]:
# IBCM model parameters
n_i_ibcm = 9  # Number of inhibitory neurons for IBCM case

# Default model rates
learnrate_ibcm = 0.001 #5e-5
tau_avg_ibcm = 200  # 2000
# Coupling changes the convergence time prediction
# because fluctuations or convergence of one neuron change the small initial
# conditions of the others and thus the escape time
# So here we do uncoupled neurons
coupling_eta_ibcm = 0.0/n_i_ibcm
ssat_ibcm = 50.0
k_c2bar_avg = 0.1
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates_default = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "linear", 
    "variant": "intrator", 
    "decay": False
}

In [ ]:
# Test run for now
# Create a default background for testing purposes
meta_seedseq = np.random.SeedSequence(0xfc3b3f6fd61dd7742adcb9e4c7f99d30)

# Choose background statistics parameters
avgnu = 1.0 / np.sqrt(n_components)
sigma2 = 0.16
epsilon_nu = 0.2  # this works even at large epsilon, 0.4 OK
tau_nu = 2.0
back_rates_default = default_nongauss_background_params(
                        n_components, avgnu, sigma2, epsilon_nu, tau_nu)

dimensions_ibcm = [n_components, n_dimensions, n_i_ibcm]

#ibcm_rates_loc, back_rates, inhib_rates_loc, 
#        options_loc, dimensions, seedseq, 
#        duration_loc=360000.0, dt_loc=1.0, skp_loc=20, full_return=False

# Run and analyze simulation derived from the meta seedsequence
all_res = run_analyze_ibcm_one_back_seed_nongauss(ibcm_rates_default, back_rates_default, inhib_rates_default, 
                        ibcm_options, dimensions_ibcm, meta_seedseq, init_ampli=0.0008,
                        duration_loc=duration, dt_loc=deltat, skp_loc=skp_default, full_returns=True)

gaps, specifs, hgamvari, hgammas_ser, ibcm_results, back_components = all_res

In [ ]:
# Visualize convergence dynamics first
plot_ibcm_results(ibcm_results, hgammas_ser, skp=skp_default, dtscale=dtscale)

In [ ]:
# Print some other convergence analysis results
print("{} out of {} odors covered".format(np.unique(specifs).size, n_components))
print("Variance of the largest hbar_gamma for each neuron:\n", hgamvari[np.arange(n_i_ibcm), specifs])

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(n_i_ibcm), gaps, marker="o", mfc="w", ms=8)
for i in range(n_i_ibcm):
    ax.annotate(str(specifs[i]), xy=(i, gaps[i]), ha="center", va="center")
ax.set_ylim([0.0, gaps.max()*1.1])
ax.set(ylabel="Alignment gap", xlabel="Neuron", xticks=np.arange(0, n_i_ibcm, 3))
plt.show()

In [ ]:
# Numerical concentration moments
conc_ser = avgnu + ibcm_results[1] + epsilon_nu * ibcm_results[1]**2
mean_conc_sim = np.mean(conc_ser)  # all odors i.i.d., can average over them. 
variance_conc_sim = np.mean((conc_ser - mean_conc_sim)**2)
thirdmoment_conc_sim = np.mean((conc_ser - mean_conc_sim)**3)

moments_conc_sim = [
    mean_conc_sim, 
    variance_conc_sim,
    thirdmoment_conc_sim
]
print(moments_conc_sim)

# Analytical prediction, exact: need moments of nu. 
variance_conc_pred = sigma2 + 2*(epsilon_nu*sigma2)**2
mean_conc_pred = avgnu + epsilon_nu*sigma2
thirdmoment_conc_pred = 6*epsilon_nu*sigma2**2 + 8*(epsilon_nu*sigma2)**3
moments_conc_pred = [
    mean_conc_pred,
    variance_conc_pred,
    thirdmoment_conc_pred
]
print(moments_conc_pred)

# Predictions of fixed point u^2
hs_hn_hd_u2 = fixedpoint_thirdmoment_exact(moments_conc_pred, 1, n_components-1)
fixed_u2 = hs_hn_hd_u2[3]

In [ ]:
variance_conc_pred**2/mean_conc_pred - thirdmoment_conc_pred

In [ ]:
# Convergence as dx/dt = x^2 of h_d in phase one, approaching the saddle point
def convergence_predict_phase1(cdsum, moments_conc, n_comp, learnrate, cosin=0.531):
    """ For vectors with exponential elements and unit-normed, cosine similarity
    is 0.516 for N_S=50 dimensions, 0.531 for N_S=25 dimensions"""
    f0 = 1 + (n_components-1) * cosin  # sum of dot products between one odor vector and all others, 
    # 1 for the vector with itself and approx. the average cosine distance between vectors of the ensemble
    # for the rest. This cosine is approx 0.6. 
    mean, vari, third = moments_conc
    aneuron = learnrate*f0*(n_comp*mean**2 + 2*vari)
    first_positive_cdsum = cdsum[cdsum > 0.0][0]
    th = 1.0 / aneuron * (1.0 / (first_positive_cdsum*mean) - 1.0)
    return th
    

In [ ]:
# Phase 2: exponential divergence away from the saddle
# along the fastest direction. We use the analytical results for the saddle point,
# which is the fixed point solution where all h_gammas are equal
# and the time scale of divergence is the largest eigenvalue of the jacobian matrix
# around that fixed point, which we find numerically.
# Unclear analytically what sets this eigenvalue, 
# but at least we can compute how long an exponential increase from initial u^2 - u^2_saddle
# takes to reach a final time
def saddle_tscale_predict_phase2(moments_conc, ibcm_rates, n_comp, back_vecs, lambd=1.0):
    """ Exponential exit time scale away from saddle point """
    # Check the saddle point where all h_gammas are equal, the model goes there at time t_d. 
    saddle_h = fixedpoint_thirdmoment_exact(moments_conc, n_components, 0, lambd=lambd_ibcm)[0]
    # Value of u^2 and h_d at the saddle
    mean_conc = moments_conc[0]
    saddle_hd = saddle_h * mean_conc * n_comp
    saddle_u2 = n_comp * saddle_h**2

    # Get the largest eigenvalue at the saddle for the divergence time scale
    specif_saddle = np.zeros(n_comp)
    jacob_saddle = jacobian_fixedpoint_thirdmoment(
                        moments_conc, ibcm_rates, specif_saddle, back_vecs, m3=1.0,
                    )
    jacob_eigvals = np.linalg.eigvals(jacob_saddle)  #eigenvalues are rates
    saddle_tscale = 1.0 / np.amax(np.real(jacob_eigvals))  # in time step units
    return saddle_tscale, [saddle_h, saddle_hd, saddle_u2]
    
def convergence_predict_phase2(tscale, u2_delta_init, u2_delta_target):
    """ Time after t_d (phase 1) to converge to u2_delta_target above saddle when starting
    at u2_delta above the saddle point value of u2. Based on setting 
    u2_delta_target = u2_delta_init * exp(t/tscale)"""
    return np.log(u2_delta_target / u2_delta_init) * tscale

In [ ]:
saddle_exit_tscale, saddlevals = saddle_tscale_predict_phase2(moments_conc_pred, 
                        ibcm_rates_default, n_components, back_components, lambd=lambd_ibcm)
saddle_h, saddle_hd, saddle_u2 = saddlevals
saddle_exit_tscale_scaled = saddle_exit_tscale * dtscale
print("Saddle exit time scale:", saddle_exit_tscale_scaled, "min")

In [ ]:
# For convergence, track sums of h_gammas within each neuron
cdsums = np.sum(hgammas_ser, axis=2)
u2sums = np.sum(hgammas_ser**2, axis=2)
tser_scaled = ibcm_results[0] * dtscale  # in min

target_u2 = 1.0 / sigma2

ncols = 5
nrows = n_i_ibcm // ncols + min(1, n_i_ibcm % ncols)
fig, axes = plt.subplots(nrows, ncols, sharex=True, sharey=True)
fig.set_size_inches(plt.rcParams["figure.figsize"][0]*0.55*ncols, 
                    plt.rcParams["figure.figsize"][1]*0.7*nrows)
colors = sns.color_palette("tab20", n_colors=n_i_ibcm*2)
tsl = slice(0, len(tser_scaled), 4)
tu_predictions = []
th_predictions = []
for i in range(n_i_ibcm):
    colors2 = colors[0], colors[1]  # Same two colors for each subplot
    axes.flat[i].plot(tser_scaled[tsl], cdsums[tsl, i], 
                 alpha=1.0, color=colors2[0], ls="-", 
                label=r"$ h_d \,/\langle c \rangle = \sum_{\gamma} h_{\gamma}$")
    axes.flat[i].plot(tser_scaled[tsl], u2sums[tsl, i], 
                 alpha=1.0, color=colors2[1], ls="--", label=r"$u^2 = \sum_{\gamma} h_{\gamma}^2$")
    axes.flat[i].set_title("Neuron {}, specif: {}".format(i, specifs[i]), y=0.9)
    #axes.flat[i].axhline(1.0 / avgnu, ls="-", color="k", lw=0.5)
    axes.flat[i].axhline(saddle_hd/mean_conc_pred, ls="-", color="k", lw=0.75)
    # saddle and final h_d are pretty much the same
    axes.flat[i].axhline(saddle_u2, ls=":", color="grey", lw=0.75)

    # u^2 at stable fixed point? Not worth showing
    #axes.flat[i].axhline(fixed_u2, ls="-.", color="grey", lw=0.75)
    
    # Prediction of convergence time?
    th_pred = convergence_predict_phase1(cdsums[:, i], moments_conc_pred, n_components, learnrate_ibcm)
    th_scaled = th_pred * dtscale
    th_scaled = min(th_scaled, 60)  # Clip to 60 minutes for plotting purposes
    th_predictions.append(th_scaled)
    axes.flat[i].axvline(th_scaled, ymax=0.9, 
        color="r", lw=0.75, ls="-", label="$t_d$ predict.")
    #axes.flat[i].axhline(0.0, ls="-", color="k", lw=0.5)
    
    # Phase 2: predict from actual t_d, using the exponential exit rate
    # obtained from the Jacobian matrix at the saddle point
    # Use for the initial value of u^2 the value at the actual t_h
    th_actual_idx = np.argmax(cdsums[:, i] > saddle_hd/mean_conc_pred)
    u2delta_at_th = u2sums[th_actual_idx, i] - saddle_u2
    u2delta_at_target = target_u2 - saddle_u2
    th_tu_pred = convergence_predict_phase2(
        saddle_exit_tscale_scaled, u2delta_at_th, u2delta_at_target)
    tu_pred = tser_scaled[th_actual_idx] + th_tu_pred
    tu_predictions.append(tu_pred)
    clr_tu = colors[3*2+1]
    axes.flat[i].axvline(tu_pred, ymax=0.9, 
        color=clr_tu, lw=0.75, ls="-", label=r"$t_u$ predict.")  # after true t_d
    axes.flat[i].axhline(target_u2, ls="--", color=clr_tu, lw=0.75)
    
    # Annotations where there is space
    if th_scaled > 25.0:
        xannot, yshiftd, yshiftu, halign = 0.0, 0.0, 0.15, "left"
    else:
        xannot, yshiftd, yshiftu, halign = 60.0, 0.5, 0.0, "right"
    axes.flat[i].annotate(r"Saddle $h_\mathrm{d}/\langle c \rangle$", 
                          xy=(xannot, saddle_hd/mean_conc_pred + yshiftd), fontsize=5, ha=halign, va="bottom")
    axes.flat[i].annotate(r"Saddle $u^2$", xy=(xannot, saddle_u2 + yshiftu), fontsize=5, 
                          ha=halign, va="top", color="grey")
    axes.flat[i].annotate(r"Target $u^2$", xy=(xannot, target_u2), fontsize=5, 
                          ha=halign, va="top", color=clr_tu)
for i in range(n_i_ibcm, axes.size):
    if i == n_i_ibcm:
        handles, labels = axes.flat[0].get_legend_handles_labels()
        handles[1], handles[0] = handles[0], handles[1]
        labels[1], labels[0] = labels[0], labels[1]
        axes.flat[i].legend(handles, labels, frameon=False, loc="upper left")
    axes.flat[i].set_axis_off()
for i in range(n_i_ibcm-ncols, n_i_ibcm):
    axes.flat[i].set_xlabel("Time (min)")
for j in range(2):
    axes[j, 0].set_ylabel(r"Sums of $h_\gamma$s")
fig.tight_layout()
plt.show()
plt.close()

# Save for final plotting

In [ ]:
bknorm_ser = l2_norm(ibcm_results[2], axis=1)
ynorm_ser = l2_norm(ibcm_results[7], axis=1)

In [ ]:
# IBCM results
fname = pj(outputs_folder, "non-gaussian_ibcm_convergence_analysis.npz")
# saddle hd, saddle u2, target u2
horiz_lines = np.asarray([saddle_hd/mean_conc_pred, saddle_u2, target_u2])
if do_save_outputs:
    np.savez_compressed(
        fname, 
        tser_scaled=tser_scaled,
        hgammas_ser=hgammas_ser,
        hgammas_specifs=specifs,
        th_predictions=np.asarray(th_predictions),
        tu_predictions=np.asarray(tu_predictions),  # actual th plus second phase duration prediction
        horiz_lines=horiz_lines,
        backnorm_ser=bknorm_ser,
        ynorm_ser=ynorm_ser,
        moments_conc=moments_conc_pred,
        fixed_point_preds=hs_hn_hd_u2
    )

In [ ]:
# BioPCA results?